In [7]:
import os

try:
    # Running as normal Python script inside src/
    this_file = os.path.abspath(__file__)
    src_root = os.path.dirname(this_file)                        # EMOTION-PRED/src
    project_root = os.path.dirname(src_root)                    # EMOTION-PRED/
except NameError:
    # Running inside Jupyter (likely src/notebooks or src/)
    cwd = os.getcwd()

    # If running inside src/notebooks → go up one level
    if cwd.endswith("notebooks"):
        src_root = os.path.abspath(os.path.join(cwd, ".."))
        project_root = os.path.dirname(src_root)
    else:
        # Running from project root directly
        project_root = cwd
        src_root = os.path.join(project_root, "src")

# Final unified paths
results_root = os.path.join(src_root, "results")
data_root = os.path.join(src_root, "data","MAMS-ACSA","raw","data_jsonl")
print(f"📂 Project root: {project_root}"
      f"\n📂 Source root: {src_root}"
      f"\n📂 Results root: {results_root}"
      f"\n📂 Data root: {data_root}")
# 3 — JSONL files
TRAIN_JSONL = os.path.join(data_root, "train.jsonl")
VAL_JSONL   = os.path.join(data_root, "val.jsonl")
TEST_JSONL  = os.path.join(data_root, "test.jsonl")
SAMPLE_JSONL = os.path.join(data_root, "sample.jsonl")
print("Using dataset directory:", data_root)



📂 Project root: /Users/hd/Desktop/EMOTION-PRED
📂 Source root: /Users/hd/Desktop/EMOTION-PRED/src
📂 Results root: /Users/hd/Desktop/EMOTION-PRED/src/results
📂 Data root: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl
Using dataset directory: /Users/hd/Desktop/EMOTION-PRED/src/data/MAMS-ACSA/raw/data_jsonl


In [8]:
GUIDELINES = """
ASPECT DEFINITIONS & CUES
-----------------------------------------
AMBIENCE (Atmosphere, Environment)
• Physical atmosphere or sensory environment: decor, lighting, music/noise, cleanliness, smell, comfort, temperature, seating, overall vibe.
• Common cues: “vibe”, “atmosphere”, “environment”, “setting”, “decor”, “layout”, “cozy”, “romantic”, “noisy”, “crowded”, “dirty”.
• Positive: lovely atmosphere, cozy.
• Negative: too loud, dirty tables.
• Neutral: dimly lit.

FOOD
• Taste, flavor, appearance, freshness, quality of dishes.
• Cues: “taste”, “flavor”, “freshness”, “undercooked”, “overcooked”, “delicious”, “gross”.
• Positive: delicious pizza.
• Negative: dry meat, awful soup.
• Neutral: factual statements about dishes.

MENU
• Menu size, variety, dietary options, availability, menu layout, confusing menus.
• Cues: “selection”, “options”, “variety”, “vegan options”, “limited”, “menu unavailable”, “confusing”.
• Positive: great variety.
• Negative: unavailable items, confusing layout.
• Neutral: normal children’s menu.

PLACE (Location)
• Location, neighborhood safety, accessibility, parking, ease of finding.
• Cues: “location”, “area”, “parking”, “hard to find”, “near station”.
• Positive: perfect location.
• Negative: unsafe area, impossible parking.
• Neutral: located next to station.

PRICE (Cost, Value for Money)
• Perception of price fairness, expensive/cheap, value for quality.
• Cues: “expensive”, “overpriced”, “cheap”, “worth it”, “value”, “rip-off”.
• Positive: good prices for portions.
• Negative: overpriced, not worth the price.
• Neutral: menu costs X.

SERVICE (Process Efficiency, Speed, Organization)
• Operational service processes: wait time, order accuracy, speed, payment, reservations. Not staff attitude.
• Cues: “slow service”, “wait time”, “efficient”, “orders messed up”.
• Positive: fast service.
• Negative: long waiting time, wrong order.
• Neutral: counter service.

STAFF (Employees, Waiters, Attitude)
• When a person is explicitly mentioned. Covers friendliness, rudeness, professionalism, helpfulness, attentiveness.
• Cues: “waiter”, “server”, “manager”, “friendly”, “rude”, “ignored”, “helpful”.
• Positive: friendly staff.
• Negative: rude waiter.
• Neutral: staff uniforms.

MISCELLANEOUS
• Overall impressions not tied to a specific aspect; general experience, overall visit, or business-level comments.
• Cues: “overall”, “experience”, “visit”, “amazing place”.
• Positive: wonderful time.
• Negative: terrible experience.
• Neutral: popular restaurant.

MENTIONED_ONLY
• Aspect is referenced but expresses no opinion, no emotion, no evaluation.
• Used only when the aspect is background info supporting another aspect.
• If ANY evaluation exists → do NOT use Mentioned_only.

MIXED EMOTIONS
• Reviewer expresses two different emotions for the same aspect category.
• Examples: “loved one dish, hated the other”, “service was helpful but frustrating”.
• Not mixed: mild variation of same polarity (“good but not great”).

-----------------------------------------
EMOTION DEFINITIONS & CUES
-----------------------------------------

ADMIRATION (Positive)
• Strong positive evaluative emotion: impressed, appreciative, respectful, elevated praise.
• Cues: “amazing”, “wow”, “impressive”, “excellent”, “stunning”, “top-notch”.
• Not admiration: weak praise (“nice”), general happiness, generic appreciation.

SATISFACTION (Positive)
• Enjoyment, comfort, pleasant surprise, happiness, relief, relaxed mood.
• Cues: “enjoyed”, “happy”, “pleasant”, “relaxing”, “surprisingly good”.
• Not satisfaction: strong admiration, gratitude, purely factual neutral.

GRATITUDE (Positive)
• Thankfulness for a specific helpful action.
• Cues: “thank you”, “grateful”, “appreciate your help”, thanking a waiter for something specific.
• Not gratitude: general appreciation of quality → admiration.

ANNOYANCE (Negative)
• Mild/moderate irritation: inconvenience, frustration, noise, crowdedness, slow service.
• Cues: “annoying”, “irritating”, “frustrating”, “bothered”, “not ideal”.
• Not annoyance: strong disappointment or disgust.

DISAPPOINTMENT (Negative)
• Unmet expectations, let-down, negative confusion.
• Cues: “disappointed”, “expected more”, “let down”, “confusing”, “sadly”, “unfortunately”.
• Not disappointment: strong anger → disgust, minor irritation → annoyance.

DISGUST (Negative)
• Strong rejection: gross food, hygiene issues, fear, unsafe environment.
• Cues: “disgusting”, “gross”, “made me sick”, “filthy”, “unsafe”, “terrifying”.
• Not disgust: mild disappointment or annoyance.

NO EMOTION (Neutral)
• Factual statements, indifference, no emotional valence.
• Cues: “okay”, “fine”, “average”, “nothing special”.
• Not neutral: weak positive “nice” → satisfaction, weak negative “not great” → disappointment.

-----------------------------------------
FINAL EMOTION SET (from PDF)
Positive: Admiration, Satisfaction, Gratitude
Negative: Disappointment, Annoyance, Disgust
Neutral: No Emotion, Mixed Emotions, Mentioned_only

-----------------------------------------
ASPECT–EMOTION PAIRS (from PDF)
ambience:
  positive: Admiration, Satisfaction
  negative: Annoyance, Disappointment
  neutral: No Emotion, Mixed Emotions, Mentioned_only

food:
  positive: Admiration, Satisfaction
  negative: Disgust, Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

menu:
  positive: Admiration, Satisfaction
  negative: Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

place:
  positive: Admiration, Satisfaction
  negative: Disappointment, Annoyance, Disgust
  neutral: No Emotion, Mixed Emotions, Mentioned_only

price:
  positive: Admiration, Satisfaction
  negative: Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

service:
  positive: Admiration, Satisfaction, Gratitude
  negative: Annoyance, Disappointment
  neutral: No Emotion, Mixed Emotions, Mentioned_only

staff:
  positive: Gratitude, Admiration, Satisfaction
  negative: Disappointment, Annoyance
  neutral: No Emotion, Mixed Emotions, Mentioned_only

miscellaneous:
  positive: Admiration, Satisfaction
  negative: Annoyance, Disappointment
  neutral: No Emotion, Mixed Emotions, Mentioned_only

  """

In [9]:
import json
import os
import re
import time
import requests
from dotenv import load_dotenv

# ======================================================
# API SETUP
# ======================================================
load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")

MODEL = "models/gemini-2.5-flash"
URL = f"https://generativelanguage.googleapis.com/v1beta/{MODEL}:generateContent"
HEADERS = {
    "Content-Type": "application/json",
    "X-goog-api-key": API_KEY
}

# ======================================================
# PATHS
# ======================================================
IN_PATH = os.path.join(data_root, "cleaned_300.jsonl")
EMOTION_JSON = os.path.join(data_root, "emotion.json")

OUT_DIR = os.path.join(results_root, "emotion-minimal")
os.makedirs(OUT_DIR, exist_ok=True)

OUT_JSON = os.path.join(OUT_DIR, "emotion_only_cleaned_300.jsonl")

# ======================================================
# LOAD DATA
# ======================================================
EMOTIONS = json.load(open(EMOTION_JSON, "r", encoding="utf-8"))
rows = [json.loads(line) for line in open(IN_PATH, "r", encoding="utf-8")]

# ======================================================
# NORMALIZE POLARITY
# ======================================================
def normalize_pol(pol):
    pol = pol.lower().strip()
    mapping = {"neagative": "negative", "postive": "positive", "nutral": "neutral"}
    return mapping.get(pol, pol)


# ======================================================
# GEMINI CALL (retry until success)
# ======================================================
def ask_gemini(prompt):
    payload = {"contents": [{"parts": [{"text": prompt}]}]}

    while True:
        start = time.time()
        try:
            r = requests.post(URL, headers=HEADERS, json=payload)
            r.raise_for_status()
            elapsed = time.time() - start
            return r.json()["candidates"][0]["content"]["parts"][0]["text"], elapsed
        except Exception as e:
            print(f"⚠️ Gemini error → retrying in 3s: {e}")
            time.sleep(3)


# ======================================================
# JSON PARSER
# ======================================================
def try_parse_json(txt):
    if txt is None:
        return None
    cleaned = txt.strip()
    cleaned = cleaned.replace("```json", "").replace("```", "").strip()
    cleaned = re.sub(r",\s*}", "}", cleaned)
    cleaned = re.sub(r",\s*]", "]", cleaned)
    try:
        return json.loads(cleaned)
    except:
        return None


# ======================================================
# BUILD MINIMAL EMOTION-ONLY PROMPT
# ======================================================
def build_prompt(review, triples):

    block = ""
    for i, t in enumerate(triples):
        asp = t["aspect"]
        pol = normalize_pol(t["polarity"])
        allowed = EMOTIONS[asp][pol]
        block += f"""
{i+1}. aspect: {asp}
   polarity: {pol}
   allowed_emotions: {allowed}
"""

    return f"""
Predict ONLY the emotions for each (aspect, polarity) pair.

### Review:
\"""{review}\"""

### Triples:
{block}

### Return STRICT JSON:
[
  {{"emotion": "..."}},
  ...
]

Rules:
- JSON only
- Number of items must match the number of triples
- Emotion must be chosen from allowed_emotions
"""


# ======================================================
# MAIN LOOP — WRITE EACH RESULT IMMEDIATELY
# ======================================================
f_out = open(OUT_JSON, "a", encoding="utf-8")
total = len(rows)

for idx, row in enumerate(rows, start=1):
    review = row["input"]
    triples = row["output"]

    prompt = build_prompt(review, triples)

    # retry until JSON is correct
    while True:
        reply, elapsed = ask_gemini(prompt)
        parsed = try_parse_json(reply)

        if isinstance(parsed, list) and len(parsed) == len(triples):
            valid = True
            for out, t in zip(parsed, triples):
                asp = t["aspect"]
                pol = normalize_pol(t["polarity"])
                allowed = EMOTIONS[asp][pol]
                emo = out.get("emotion", "").strip().capitalize()
                if emo not in allowed:
                    valid = False
                    break

            if valid:
                break

        print(f"❌ Invalid JSON or structure at row {idx}. Retrying…")
        time.sleep(2)

    # Build final clean output
    final_objs = []
    for t, out in zip(triples, parsed):
        asp = t["aspect"]
        pol = normalize_pol(t["polarity"])
        emo = out["emotion"].strip().capitalize()
        final_objs.append({
            "aspect": asp,
            "polarity": pol,
            "emotion": emo
        })

    # Write IMMEDIATELY
    f_out.write(json.dumps({"input": review, "output": final_objs}, ensure_ascii=False) + "\n")

    print(f"✔ [{idx}/{total}] OK — {len(triples)} emotions — {elapsed:.1f}s")

f_out.close()
print("\nDONE →", OUT_JSON)

✔ [1/302] OK — 2 emotions — 6.3s
✔ [2/302] OK — 2 emotions — 3.5s
✔ [3/302] OK — 2 emotions — 3.5s
✔ [4/302] OK — 1 emotions — 1.9s
✔ [5/302] OK — 3 emotions — 2.9s
✔ [6/302] OK — 2 emotions — 4.1s
✔ [7/302] OK — 1 emotions — 1.8s
✔ [8/302] OK — 1 emotions — 2.7s
✔ [9/302] OK — 2 emotions — 2.3s
✔ [10/302] OK — 1 emotions — 4.9s
✔ [11/302] OK — 2 emotions — 2.5s
✔ [12/302] OK — 2 emotions — 3.1s
✔ [13/302] OK — 3 emotions — 4.3s
✔ [14/302] OK — 2 emotions — 4.0s
✔ [15/302] OK — 2 emotions — 7.7s
✔ [16/302] OK — 2 emotions — 1.5s
✔ [17/302] OK — 3 emotions — 4.8s
✔ [18/302] OK — 1 emotions — 2.4s
✔ [19/302] OK — 2 emotions — 2.6s
✔ [20/302] OK — 3 emotions — 4.6s
✔ [21/302] OK — 2 emotions — 2.5s
✔ [22/302] OK — 1 emotions — 2.5s
✔ [23/302] OK — 2 emotions — 2.7s
✔ [24/302] OK — 3 emotions — 4.4s
✔ [25/302] OK — 2 emotions — 8.8s
✔ [26/302] OK — 2 emotions — 3.2s
✔ [27/302] OK — 1 emotions — 2.0s
✔ [28/302] OK — 2 emotions — 3.7s
✔ [29/302] OK — 2 emotions — 2.2s
✔ [30/302] OK — 3 emoti